# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary classifier.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use a random forest regressor, as well as another classifier of your choice; either logistic regression, SVM, or KNN. 

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

In [359]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib

In [360]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer

## Loop the scraping sites into the function loop

In [361]:
ny61_80 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2461k-%2480k&radius=25&l=New+York&fromage=any&limit=200&sort=&psf=advsrch'
ny81_100 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2481%2C000+-+%24100%2C000&radius=25&l=New+York&fromage=any&limit=200&sort=&psf=advsrch'
ny101_120 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24101%2C000+-+%24120%2C000&radius=25&l=New+York&fromage=any&limit=200&sort=&psf=advsrch'
ny121_140 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24121%2C000+-+%24140%2C000&radius=25&l=New+York&fromage=any&limit=200&sort=&psf=advsrch'
ny141_160 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24141%2C000+-+%24160%2C000&radius=25&l=New+York&fromage=any&limit=200&sort=&psf=advsrch'
ny161_180 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24161%2C000+-+%24180%2C000&radius=25&l=New+York&fromage=any&limit=200&sort=&psf=advsrch'
ny181_200 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24181K-%24200K&radius=25&l=New+York&fromage=any&limit=200&sort=&psf=advsrch'

sf61_80 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2461k-%2480k&radius=25&l=San+Francisco%2C+CA&fromage=any&limit=200&sort=&psf=advsrch'
sf81_100 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2481%2C000+-+%24100%2C000&radius=25&l=San+Francisco%2C+CA&fromage=any&limit=200&sort=&psf=advsrch'
sf101_120 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24101%2C000+-+%24120%2C000&radius=25&l=San+Francisco%2C+CA&fromage=any&limit=200&sort=&psf=advsrch'
sf121_140 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24121%2C000+-+%24140%2C000&radius=25&l=San+Francisco%2C+CA&fromage=any&limit=200&sort=&psf=advsrch'
sf141_160 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24141%2C000+-+%24160%2C000&radius=25&l=San+Francisco%2C+CA&fromage=any&limit=200&sort=&psf=advsrch'
sf161_180 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24161%2C000+-+%24180%2C000&radius=25&l=San+Francisco%2C+CA&fromage=any&limit=200&sort=&psf=advsrch'
sf181_200 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24181K-%24200K&radius=25&l=San+Francisco%2C+CA&fromage=any&limit=200&sort=&psf=advsrch'

dc61_80 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2461k-%2480k&radius=25&l=Washington+City%2C+DC&fromage=any&limit=200&sort=&psf=advsrch'
dc81_100 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2481%2C000+-+%24100%2C000&radius=25&l=Washington+City%2C+DC&fromage=any&limit=200&sort=&psf=advsrch'
dc101_120 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24101%2C000+-+%24120%2C000&radius=25&l=Washington+City%2C+DC&fromage=any&limit=200&sort=&psf=advsrch'
dc121_140 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24121%2C000+-+%24140%2C000&radius=25&l=Washington+City%2C+DC&fromage=any&limit=200&sort=&psf=advsrch'
dc141_160 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24141%2C000+-+%24160%2C000&radius=25&l=Washington+City%2C+DC&fromage=any&limit=200&sort=&psf=advsrch'
dc161_180 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24161%2C000+-+%24180%2C000&radius=25&l=Washington+City%2C+DC&fromage=any&limit=200&sort=&psf=advsrch'
dc181_200 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24181K-%24200K&radius=25&l=Washington+City%2C+DC&fromage=any&limit=200&sort=&psf=advsrch'

bos61_80 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2461k-%2480k&radius=25&l=Boston%2C+MA&fromage=any&limit=200&sort=&psf=advsrch'
bos81_100 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2481%2C000+-+%24100%2C000&radius=25&l=Boston%2C+MA&fromage=any&limit=200&sort=&psf=advsrch'
bos101_120 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24101%2C000+-+%24120%2C000&radius=25&l=Boston%2C+MA&fromage=any&limit=200&sort=&psf=advsrch'
bos121_140 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24121%2C000+-+%24140%2C000&radius=25&l=Boston%2C+MA&fromage=any&limit=200&sort=&psf=advsrch'
bos141_160 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24141%2C000+-+%24160%2C000&radius=25&l=Boston%2C+MA&fromage=any&limit=200&sort=&psf=advsrch'
bos161_180 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24161%2C000+-+%24180%2C000&radius=25&l=Boston%2C+MA&fromage=any&limit=200&sort=&psf=advsrch'
bos181_200 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24181K-%24200K&radius=25&l=Boston%2C+MA&fromage=any&limit=200&sort=&psf=advsrch'

hou61_80 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2461k-%2480k&radius=25&l=Houston%2C+TX&fromage=any&limit=200&sort=&psf=advsrch'
hou81_100 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2481%2C000+-+%24100%2C000&radius=25&l=Houston%2C+TX&fromage=any&limit=200&sort=&psf=advsrch'
hou101_120 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24101%2C000+-+%24120%2C000&radius=25&l=Houston%2C+TX&fromage=any&limit=200&sort=&psf=advsrch'
hou121_140 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24121%2C000+-+%24140%2C000&radius=25&l=Houston%2C+TX&fromage=any&limit=200&sort=&psf=advsrch'
hou141_160 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24141%2C000+-+%24160%2C000&radius=25&l=Houston%2C+TX&fromage=any&limit=200&sort=&psf=advsrch'
hou161_180 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24161%2C000+-+%24180%2C000&radius=25&l=Houston%2C+TX&fromage=any&limit=200&sort=&psf=advsrch'
hou181_200 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24181K-%24200K&radius=25&l=Houston%2C+TX&fromage=any&limit=200&sort=&psf=advsrch'

chi61_80 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2461k-%2480k&radius=25&l=Chicago%2C+IL&fromage=any&limit=200&sort=&psf=advsrch'
chi81_100 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%2481%2C000+-+%24100%2C000&radius=25&l=Chicago%2C+IL&fromage=any&limit=200&sort=&psf=advsrch'
chi101_120 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24101%2C000+-+%24120%2C000&radius=25&l=Chicago%2C+IL&fromage=any&limit=200&sort=&psf=advsrch'
chi121_140 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24121%2C000+-+%24140%2C000&radius=25&l=Chicago%2C+IL&fromage=any&limit=200&sort=&psf=advsrch'
chi141_160 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24141%2C000+-+%24160%2C000&radius=25&l=Chicago%2C+IL&fromage=any&limit=200&sort=&psf=advsrch'
chi161_180 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24161%2C000+-+%24180%2C000&radius=25&l=Chicago%2C+IL&fromage=any&limit=200&sort=&psf=advsrch'
chi181_200 = 'https://www.indeed.com/jobs?as_and=data+scientist&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&salary=%24181K-%24200K&radius=25&l=Chicago%2C+IL&fromage=any&limit=200&sort=&psf=advsrch'


While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Write 4 functions to extract each item: location, company, job, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```

##### - Make sure these functions are robust and can handle cases where the data/field may not be available.
>- Remember to check if a field is empty or None for attempting to call methods on it
>- Remember to use try/except if you anticipate errors.

- **Test** the functions on the results above and simple examples

### Create DataFrame

In [362]:
df = pd.DataFrame(columns = [['Title','Location', 'City', 'Company', 'Salary_Range', 'Salary_Avg', 'Summary']])

### Scrape Everything

In [363]:
def scrape_all(url, city, sal_range, sal_average):
    html = urllib.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")
    ## locate the job posting block
    for i in soup.find_all('div',{'class':' row result'}):
        title = i.find('a').text
        location = i.find('span',{'class':'location'}).text
        company = i.find('span',{'class':'company'}).text
        salary_range = sal_range
        salary_avg = sal_average
        summary = i.find('span',{'class':'summary'}).text
        df.loc[len(df)] = [title,location, city, company,salary_range,salary_avg,summary]

In [364]:
scrape_all(ny61_80, 'NY', '61k-80k', 70.5)
scrape_all(ny81_100, 'NY', '81k-100k', 90.5)
scrape_all(ny101_120, 'NY', '101k-120k', 110.5)
scrape_all(ny121_140, 'NY', '121k-140k', 130.5)
scrape_all(ny141_160, 'NY', '141k-160k', 150.5)
scrape_all(ny161_180, 'NY', '161k-180k', 170.5)
scrape_all(ny181_200, 'NY', '181k-200k', 190.5)

In [365]:
scrape_all(sf61_80, 'SF', '61k-80k', 70.5)
scrape_all(sf81_100, 'SF', '81k-100k', 90.5)
scrape_all(sf101_120, 'SF', '101k-120k', 110.5)
scrape_all(sf121_140, 'SF', '121k-140k', 130.5)
scrape_all(sf141_160, 'SF', '141k-160k', 150.5)
scrape_all(sf161_180, 'SF', '161k-180k', 170.5)
scrape_all(sf181_200, 'SF', '181k-200k', 190.5)

In [366]:
scrape_all(dc61_80, 'DC', '61k-80k', 70.5)
scrape_all(dc81_100, 'DC', '81k-100k', 90.5)
scrape_all(dc101_120, 'DC', '101k-120k', 110.5)
scrape_all(dc121_140, 'DC', '121k-140k', 130.5)
scrape_all(dc141_160, 'DC', '141k-160k', 150.5)
scrape_all(dc161_180, 'DC', '161k-180k', 170.5)
scrape_all(dc181_200, 'DC', '181k-200k', 190.5)

In [367]:
scrape_all(bos61_80, 'BOS', '61k-80k', 70.5)
scrape_all(bos81_100, 'BOS', '81k-100k', 90.5)
scrape_all(bos101_120, 'BOS', '101k-120k', 110.5)
scrape_all(bos121_140, 'BOS', '121k-140k', 130.5)
scrape_all(bos141_160, 'BOS', '141k-160k', 150.5)
scrape_all(bos161_180, 'BOS', '161k-180k', 170.5)
scrape_all(bos181_200, 'BOS', '181k-200k', 190.5)

In [368]:
scrape_all(hou61_80, 'HOU', '61k-80k', 70.5)
scrape_all(hou81_100, 'HOU', '81k-100k', 90.5)
scrape_all(hou101_120, 'HOU', '101k-120k', 110.5)
scrape_all(hou121_140, 'HOU', '121k-140k', 130.5)
scrape_all(hou141_160, 'HOU', '141k-160k', 150.5)
scrape_all(hou161_180, 'HOU', '161k-180k', 170.5)
scrape_all(hou181_200, 'HOU', '181k-200k', 190.5)

In [369]:
scrape_all(chi61_80, 'CHI', '61k-80k', 70.5)
scrape_all(chi81_100, 'CHI', '81k-100k', 90.5)
scrape_all(chi101_120, 'CHI', '101k-120k', 110.5)
scrape_all(chi121_140, 'CHI', '121k-140k', 130.5)
scrape_all(chi141_160, 'CHI', '141k-160k', 150.5)
scrape_all(chi161_180, 'CHI', '161k-180k', 170.5)
scrape_all(chi181_200, 'CHI', '181k-200k', 190.5)

In [370]:
df['Company'] = df['Company'].str.strip('\n')
df['Summary'] = df['Summary'].str.strip('\n')

In [371]:
df.tail()

,Title,Location,City,Company,Salary_Range,Salary_Avg,Summary
2316,"Analytics Go-To-Market Account Lead, Retail In...","Chicago, IL",CHI,Accenture,141k-160k,150.5,Understanding of analytics roles and responsib...
2317,Senior Engineer - Site Reliability,"Chicago, IL",CHI,Civis Analytics,141k-160k,150.5,Engineers collaborate across departments with ...
2318,DevOps Engineer,"Evanston, IL 60201",CHI,Jobspring Partners,141k-160k,150.5,The team is taking a concept that they’ve used...
2319,Data Scientists and Quantitative Researchers,"Chicago, IL",CHI,Optiver,161k-180k,170.5,Data Scientists and Quantitative Researchers. ...
2320,Vice President - Global Digital-Analytics,"Chicago, IL",CHI,Capgemini,161k-180k,170.5,"Insights & Data. (big data, enterprise content..."


Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [372]:
df.drop_duplicates(inplace=True)

In [373]:
df.tail()

,Title,Location,City,Company,Salary_Range,Salary_Avg,Summary
2316,"Analytics Go-To-Market Account Lead, Retail In...","Chicago, IL",CHI,Accenture,141k-160k,150.5,Understanding of analytics roles and responsib...
2317,Senior Engineer - Site Reliability,"Chicago, IL",CHI,Civis Analytics,141k-160k,150.5,Engineers collaborate across departments with ...
2318,DevOps Engineer,"Evanston, IL 60201",CHI,Jobspring Partners,141k-160k,150.5,The team is taking a concept that they’ve used...
2319,Data Scientists and Quantitative Researchers,"Chicago, IL",CHI,Optiver,161k-180k,170.5,Data Scientists and Quantitative Researchers. ...
2320,Vice President - Global Digital-Analytics,"Chicago, IL",CHI,Capgemini,161k-180k,170.5,"Insights & Data. (big data, enterprise content..."


In [374]:
df.reset_index

<bound method DataFrame.reset_index of                                                   Title  \
0                                          STATISTICIAN   
1                                        Data Scientist   
2                        Disaster Recovery Data Analyst   
3                               Data Science Internship   
4                                  Quantitative Analyst   
5                             Bike Share Data Scientist   
6                           Manager of Statistical Data   
7                             Junior Research Scientist   
8                               Senior Research Analyst   
9                                    Sr. Data Scientist   
10                                Research Data Manager   
11                                         Statistician   
12                                  Statistical Analyst   
13         Quantitative Primary Market Research Analyst   
14                                       Data Scientist   
15               

In [375]:
df = pd.concat([df,pd.get_dummies(df.City)], axis=1)

In [376]:
df.head(2)

,Title,Location,City,Company,Salary_Range,Salary_Avg,Summary,BOS,CHI,DC,HOU,NY,SF
0,STATISTICIAN,"New York, NY",NY,United Nations,61k-80k,70.5,"Organizes, designs, plans and carries out the ...",0.0,0.0,0.0,0.0,1.0,0.0
1,Data Scientist,"New York, NY",NY,Rodale Inc.,61k-80k,70.5,"Query & analyze data:. Data analysis, visualiz...",0.0,0.0,0.0,0.0,1.0,0.0


In [377]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Save your results as a CSV

In [381]:
# Export to csv
df.to_csv('indeed_salaries.csv', encoding='utf-8')

End of Scraping

# Modeling

## Predicting salaries using Random Forests + Another Classifier

#### Load in the the data of scraped salaries

In [396]:
df = pd.read_csv('indeed_salaries.csv')

In [397]:
df.head(3)

,Unnamed: 0,Title,Location,City,Company,Salary_Range,Salary_Avg,Summary,BOS,CHI,DC,HOU,NY,SF
0,0,STATISTICIAN,"New York, NY",NY,United Nations,61k-80k,70.5,"Organizes, designs, plans and carries out the ...",0.0,0.0,0.0,0.0,1.0,0.0
1,1,Data Scientist,"New York, NY",NY,Rodale Inc.,61k-80k,70.5,"Query & analyze data:. Data analysis, visualiz...",0.0,0.0,0.0,0.0,1.0,0.0
2,2,Disaster Recovery Data Analyst,"New York, NY",NY,ICF International,61k-80k,70.5,Disaster Recovery Data Analyst. Organize data ...,0.0,0.0,0.0,0.0,1.0,0.0


In [398]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [399]:
df.tail(3)

,Title,Location,City,Company,Salary_Range,Salary_Avg,Summary,BOS,CHI,DC,HOU,NY,SF
2318,DevOps Engineer,"Evanston, IL 60201",CHI,Jobspring Partners,141k-160k,150.5,The team is taking a concept that they’ve used...,0.0,1.0,0.0,0.0,0.0,0.0
2319,Data Scientists and Quantitative Researchers,"Chicago, IL",CHI,Optiver,161k-180k,170.5,Data Scientists and Quantitative Researchers. ...,0.0,1.0,0.0,0.0,0.0,0.0
2320,Vice President - Global Digital-Analytics,"Chicago, IL",CHI,Capgemini,161k-180k,170.5,"Insights & Data. (big data, enterprise content...",0.0,1.0,0.0,0.0,0.0,0.0


In [464]:
bos_salary = df[df.BOS==1].Salary_Avg.mean()
print bos_salary

105.282608696


In [457]:
chi_salary = df[df.CHI==1].Salary_Avg.mean()
print chi_salary

100.813390313


In [458]:
dc_salary = df[df.DC==1].Salary_Avg.mean()
print dc_salary

104.439393939


In [459]:
hou_salary = df[df.HOU==1].Salary_Avg.mean()
print hou_salary

95.5


In [460]:
ny_salary = df[df.NY==1].Salary_Avg.mean()
print ny_salary

113.202702703


In [482]:
sf_salary = df[df.SF==1].Salary_Avg.mean()
print sf_salary

111.804347826


#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choice the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [400]:
mean_salary = np.mean(df.Salary_Avg)
print mean_salary

107.303102111


In [401]:
np.median(df.Salary_Avg)

110.5

### mapping salary ranges

    0 if below the median
    1 if equal to or above the median

In [402]:
df['Above_Median'] = df['Salary_Range'].map({'61k-80k':0,'81k-100k':0,'101k-120k':1, '121k-140k':1, '141k-160k':1, '161k-180k':1, '181k:200k':1})

In [403]:
df['Above_Median'].value_counts()

1.0    1289
0.0    1031
Name: Above_Median, dtype: int64

In [404]:
df.tail(2)

,Title,Location,City,Company,Salary_Range,Salary_Avg,Summary,BOS,CHI,DC,HOU,NY,SF,Above_Median
2319,Data Scientists and Quantitative Researchers,"Chicago, IL",CHI,Optiver,161k-180k,170.5,Data Scientists and Quantitative Researchers. ...,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2320,Vice President - Global Digital-Analytics,"Chicago, IL",CHI,Capgemini,161k-180k,170.5,"Insights & Data. (big data, enterprise content...",0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [405]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [406]:
def run_metrics(model, X_train,y_train,y_test,y_pred):
    crossvalscore = cross_val_score(model, X_train, y_train).mean()
    accuracyscore = accuracy_score(y_test,y_pred)
    classificationreport = classification_report(y_test,y_pred)
    confusionmatrix = confusion_matrix(y_test,y_pred)
    print 'Cross_Val_Score: ' + str(crossvalscore) + '\n'
    print 'Accuracy_Score: ' + str(accuracyscore) + '\n'
    print 'Classification_Report: \n', str(classificationreport)+ '\n'
    print 'Confusion_Matrix: ' + '\n' + str(confusionmatrix) + '\n'

#### Thought experiment: What is the baseline accuracy for this model?

In [407]:
len(df[df.Above_Median == 0])

1031

In [408]:
len(df[df.Above_Median == 1])

1289

In [409]:
1286.0/(1027+1286)

0.5559878945092953

### Baseline Model Score

The baseline accuracy of my model is 56% because my greatest class (= or > the median) contains 56% of my points. Thus, if I guessed that class every time, I could score 56%. For this reason, my model must perform better than 56% to have any value.

In [410]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


## Random Forest Using Cities Only

In [411]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [412]:
X_city = df[['BOS', 'CHI','DC','HOU','NY','SF']].apply(LabelEncoder().fit_transform)
y_city = le.fit_transform(df['Above_Median'])

In [413]:
X_train_city,X_test_city,y_train_city,y_test_city = train_test_split(X_city,y_city,random_state=99)

In [414]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train_city,y_train_city)
rfc_pred_city = rfc.predict(X_test_city)

run_metrics(rfc, X_train_city,y_train_city,y_test_city, rfc_pred_city)

Cross_Val_Score: 0.582171536421

Accuracy_Score: 0.555938037866

Classification_Report: 
             precision    recall  f1-score   support

          0       0.49      0.21      0.29       256
          1       0.57      0.83      0.68       325

avg / total       0.54      0.56      0.51       581


Confusion_Matrix: 
[[ 53 203]
 [ 55 270]]



## Random Forest Regressor

Let's try treating this as a regression problem. 

- Train a random forest regressor on the regression problem and predict your dependent.
- Evaluate the score with a 5-fold cross-validation
- Do a scatter plot of the predicted vs actual scores for each of the 5 folds, do they match?

In [415]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier

In [416]:
le = LabelEncoder()
X_reg = df[['BOS', 'CHI','DC','HOU','NY','SF']].apply(le.fit_transform)
y_reg = le.fit_transform(df['Salary_Avg'])

X_train_reg,X_test_reg,y_train_reg,y_test_reg = train_test_split(X_reg,y_reg,random_state=99)

In [483]:
rfr = RandomForestRegressor()
rfr.fit(X_train_reg,y_train_reg)
rfr_pred = rfr.predict(X_test_reg)
print cross_val_score(rfr, X_train_reg, y_train_reg).mean()

0.0367509500432


## CVEC

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [419]:
cvec = CountVectorizer(stop_words='english')
cvec.fit(df.Summary)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [420]:
df_nlp_summary = pd.DataFrame(cvec.transform(df.Summary).todense(),
             columns=cvec.get_feature_names())

In [421]:
df_nlp_summary.head(2)

,000,04,10,100,1000,1010data,11,110,12,12066,...,younger,yume,zero,zoomdata,企业不同,但跟大多数consulting,全球范围内员工超过2500人,在科技服务方面成军20年,虽然对于国际学生部分都是consulting业务,ﬁll
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [422]:
df_nlp_summary.transpose().sort_values(0, ascending=False).head(10).transpose()

,collection,selecting,evaluation,plans,methods,data,designs,dissemination,carries,organizes
0,1,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,3,0,0,0,0
2,0,0,0,0,0,3,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,2,0,0,0,0
6,0,0,0,0,0,2,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,3,0,0,0,0


In [423]:
df_nlp_summary.sum().sort_values(ascending=False)

data                 4115
scientist             823
scientists            613
experience            407
analysis              380
team                  376
research              288
work                  250
science               236
analytics             222
engineers             199
learning              181
machine               178
looking               176
big                   174
working               163
senior                152
new                   150
statistical           146
management            146
join                  142
seeking               139
business              135
large                 134
develop               129
software              124
using                 119
sets                  117
including             113
analyst               113
                     ... 
exceptions              1
preferable              1
estimates               1
estimation              1
et                      1
priority                1
ethical                 1
ethos       

## Random Forest with No Cities and Hand Selected NLP Words

In [468]:
le = LabelEncoder()
X_nlp = df_nlp_summary[['junior', 'senior', 'manager', 'masters', 'phd', 'entry', 'scientist', 'machine','research','software']].apply(le.fit_transform)
y_nlp = le.fit_transform(df['Above_Median'])

In [469]:
X_train_nlp,X_test_nlp,y_train_nlp,y_test_nlp = train_test_split(X_nlp,y_nlp,random_state=99)

In [470]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train_nlp,y_train_nlp)
rfc_pred_nlp = rfc.predict(X_test_nlp)

run_metrics(rfc, X_train_nlp,y_train_nlp,y_test_nlp, rfc_pred_nlp)

Cross_Val_Score: 0.58048299889

Accuracy_Score: 0.60413080895

Classification_Report: 
             precision    recall  f1-score   support

          0       0.54      0.74      0.62       256
          1       0.71      0.50      0.58       325

avg / total       0.63      0.60      0.60       581


Confusion_Matrix: 
[[189  67]
 [163 162]]



In [428]:
df_all = pd.concat([df,df_nlp_summary],axis=1)

In [429]:
df_all.head(3)

,Title,Location,City,Company,Salary_Range,Salary_Avg,Summary,BOS,CHI,DC,...,younger,yume,zero,zoomdata,企业不同,但跟大多数consulting,全球范围内员工超过2500人,在科技服务方面成军20年,虽然对于国际学生部分都是consulting业务,ﬁll
0,STATISTICIAN,"New York, NY",NY,United Nations,61k-80k,70.5,"Organizes, designs, plans and carries out the ...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Data Scientist,"New York, NY",NY,Rodale Inc.,61k-80k,70.5,"Query & analyze data:. Data analysis, visualiz...",0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,Disaster Recovery Data Analyst,"New York, NY",NY,ICF International,61k-80k,70.5,Disaster Recovery Data Analyst. Organize data ...,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


## Random Forest with Cities and Hand Selected NLP Words

In [430]:
le = LabelEncoder()
X_choice = df_all[['Title','BOS','CHI','DC','HOU','NY','SF','junior', 'senior', 'masters', 'phd', 'entry', 'scientist', 'machine']].apply(LabelEncoder().fit_transform)
y_choice = le.fit_transform(df_all['Above_Median'])
X_train_choice,X_test_choice,y_train_choice,y_test_choice = train_test_split(X_choice,y_choice,random_state=99)

In [443]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train_choice,y_train_choice)
rfc_pred_choice = rfc.predict(X_test_choice)

run_metrics(rfc, X_train_choice,y_train_choice,y_test_choice, rfc_pred_choice)

Cross_Val_Score: 0.639086231149

Accuracy_Score: 0.638554216867

Classification_Report: 
             precision    recall  f1-score   support

          0       0.58      0.65      0.61       256
          1       0.70      0.63      0.66       325

avg / total       0.64      0.64      0.64       581


Confusion_Matrix: 
[[167  89]
 [121 204]]



## Random Forest with All Features

In [432]:
X_all = df_all.drop(['Above_Median','Salary_Range','Salary_Avg'], axis=1).apply(LabelEncoder().fit_transform)
y_all = le.fit_transform(df_all['Above_Median'])
X_train_all,X_test_all,y_train_all,y_test_all = train_test_split(X_all,y_all,random_state=99)

In [433]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train_all,y_train_all)
rfc_pred_all = rfc.predict(X_test_all)
score = cross_val_score(rfc, X_train_all, y_train_all).mean()

In [434]:
run_metrics(rfc, X_train_all,y_train_all,y_test_all, rfc_pred_all)

Cross_Val_Score: 0.650609238585

Accuracy_Score: 0.719449225473

Classification_Report: 
             precision    recall  f1-score   support

          0       0.69      0.67      0.68       256
          1       0.74      0.76      0.75       325

avg / total       0.72      0.72      0.72       581


Confusion_Matrix: 
[[171  85]
 [ 78 247]]



## Random Forest with NLP on Title Only

In [435]:
cvec = CountVectorizer(stop_words='english')
cvec.fit(df.Title)
df_nlp_Title = pd.DataFrame(cvec.transform(df.Summary).todense(),
             columns=cvec.get_feature_names())

In [436]:
X_title = df_nlp_Title.apply(LabelEncoder().fit_transform)
y_title = le.fit_transform(df_all['Above_Median'])
X_train_title,X_test_title,y_train_title,y_test_title = train_test_split(X_title,y_title,random_state=99)

In [437]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train_title,y_train_title)
rfc_pred_title = rfc.predict(X_test_title)

run_metrics(rfc,X_train_title,y_train_title, y_test_title,rfc_pred_title)

Cross_Val_Score: 0.664927680739

Accuracy_Score: 0.717728055077

Classification_Report: 
             precision    recall  f1-score   support

          0       0.67      0.70      0.69       256
          1       0.76      0.73      0.74       325

avg / total       0.72      0.72      0.72       581


Confusion_Matrix: 
[[179  77]
 [ 87 238]]



## Logistic Regression on Summary CVEC and City

In [438]:
X_all = df_all.drop(['Above_Median','Salary_Range','Salary_Avg'], axis=1).apply(LabelEncoder().fit_transform)
y_all = le.fit_transform(df_all['Above_Median'])
X_train_all,X_test_all,y_train_all,y_test_all = train_test_split(X_all,y_all,random_state=99)

In [439]:
logit = LogisticRegression()
logit.fit(X_train_all,y_train_all)
logit_pred_all = logit.predict(X_test_all)

run_metrics(logit,X_train_all,y_train_all, y_test_all,logit_pred_all)

Cross_Val_Score: 0.700051635418

Accuracy_Score: 0.73321858864

Classification_Report: 
             precision    recall  f1-score   support

          0       0.73      0.62      0.67       256
          1       0.73      0.82      0.78       325

avg / total       0.73      0.73      0.73       581


Confusion_Matrix: 
[[159  97]
 [ 58 267]]



### Gradient Boosting with City and CVEC

In [441]:
gb = GradientBoostingClassifier()
gb.fit(X_train_all,y_train_all)
gb_pred_all = gb.predict(X_test_all)

run_metrics(gb,X_train_all,y_train_all, y_test_all,gb_pred_all)

Cross_Val_Score: 0.691959118993

Accuracy_Score: 0.729776247849

Classification_Report: 
             precision    recall  f1-score   support

          0       0.73      0.61      0.67       256
          1       0.73      0.82      0.77       325

avg / total       0.73      0.73      0.73       581


Confusion_Matrix: 
[[157  99]
 [ 58 267]]



In [444]:
features = pd.DataFrame(gb.feature_importances_, index=df_all.drop(['Above_Median','Salary_Range','Salary_Avg'],axis=1).columns, columns=['importance'])
print features.sort_values(['importance'], ascending=False).head(35)

              importance
compensation    0.099947
199             0.087106
gaining         0.076913
Title           0.058574
Company         0.039662
data            0.017436
Location        0.017388
lead            0.010367
scientists      0.010307
scientist       0.010291
analyst         0.010147
machine         0.009662
senior          0.008827
Summary         0.008802
interpret       0.008490
analytics       0.008254
research        0.008116
big             0.008059
environments    0.007980
analysis        0.007890
building        0.007428
collection      0.007196
CHI             0.007024
science         0.006613
using           0.006214
experience      0.006025
models          0.005960
analyzing       0.005753
analyze         0.005702
risk            0.005570
analysts        0.005436
geospatial      0.005297
reports         0.005071
brand           0.004989
implement       0.004814


## TFIDF

In [337]:
tvec = TfidfVectorizer(stop_words='english')
tvec.fit(df.Summary)

df_tfidf = pd.DataFrame(tvec.transform(df.Summary).todense(),
                   columns=tvec.get_feature_names())

df_tfidf.head()

,000,04,10,100,1000,1010data,11,110,12066,1300,...,younger,yume,zero,zoomdata,企业不同,但跟大多数consulting,全球范围内员工超过2500人,在科技服务方面成军20年,虽然对于国际学生部分都是consulting业务,ﬁll
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Gradient Boosting with TFIDF Only

In [352]:
X_tfidf = df_tfidf.apply(LabelEncoder().fit_transform)
y_tfidf = le.fit_transform(df_all['Above_Median'])
X_train_tfidf,X_test_tfidf,y_train_tfidf,y_test_tfidf = train_test_split(X_tfidf,y_tfidf,random_state=99)

In [341]:
gb = gb.fit(X_train_tfidf,y_train_tfidf)
gb_pred_tfidf = gb.predict(X_test_tfidf)

run_metrics(gb,X_train_tfidf,y_train_tfidf, y_test_tfidf,gb_pred_tfidf)

Cross_Val_Score: 0.681270188249

Accuracy_Score: 0.721934369603

Classification_Report: 
             precision    recall  f1-score   support

          0       0.72      0.61      0.66       257
          1       0.72      0.81      0.76       322

avg / total       0.72      0.72      0.72       579


Confusion_Matrix: 
[[156 101]
 [ 60 262]]



In [342]:
df.head()

,Title,Location,City,Company,Salary_Range,Salary_Avg,Summary,Above_Median,BOS,CHI,DC,HOU,NY,SF
0,Data Scientist / Data Science,"New York, NY",NY,Corporate Technology,61k-80k,70.5,JOB DESCRIPTION – DATA SCIENTIST. We're lookin...,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Senior Research Scientist,"New York, NY",NY,New York University,61k-80k,70.5,Senior Research Scientist*. The McDevitt lab i...,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Associate Research Scientist,"New York, NY",NY,New York University,61k-80k,70.5,Associate Research Scientist*. The Data Scient...,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Data Scientist,"New York, NY",NY,Rodale Inc.,61k-80k,70.5,"Query & analyze data:. Data analysis, visualiz...",0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,STATISTICIAN,"New York, NY",NY,United Nations,61k-80k,70.5,"Organizes, designs, plans and carries out the ...",0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [347]:
df_tfidf_city = pd.concat([df,df_tfidf],axis=1)

In [349]:
df_tfidf_city.drop(['Location','City','Salary_Range','Salary_Avg','Summary'], axis=1, inplace=True)

## Gradient Boosting with TFIDF and City

In [476]:
X_tfidf_city = df_tfidf_city.drop(['Above_Median'], axis=1).apply(LabelEncoder().fit_transform)
y_tfidf_city = le.fit_transform(df_tfidf_city['Above_Median'])
X_train_tfidfcity,X_test_tfidfcity,y_train_tfidfcity,y_test_tfidfcity = train_test_split(X_tfidf_city,y_tfidf_city,random_state=99)

In [477]:
gb = gb.fit(X_train_tfidfcity,y_train_tfidfcity)
gb_pred_tfidfcity = gb.predict(X_test_tfidfcity)

run_metrics(gb,X_train_tfidfcity,y_train_tfidfcity, y_test_tfidfcity,gb_pred_tfidfcity)

Cross_Val_Score: 0.696275436221

Accuracy_Score: 0.72884283247

Classification_Report: 
             precision    recall  f1-score   support

          0       0.72      0.65      0.68       257
          1       0.74      0.80      0.77       322

avg / total       0.73      0.73      0.73       579


Confusion_Matrix: 
[[166  91]
 [ 66 256]]



In [478]:
features = pd.DataFrame(gb.feature_importances_, index=df_tfidf_city.drop(['Above_Median'],axis=1).columns, columns=['importance'])
print features.sort_values(['importance'], ascending=False).head(35)

               importance
compensation     0.106378
gaining          0.074890
199              0.071608
data             0.038454
Title            0.034351
Company          0.031081
scientists       0.018922
science          0.016820
analysis         0.013777
databases        0.011802
research         0.011366
analyst          0.010439
lead             0.009307
analytics        0.009135
reports          0.008207
analyzing        0.007941
risk             0.007921
scientist        0.007617
senior           0.007484
help             0.007431
skills           0.007251
engineer         0.007226
big              0.007172
software         0.006847
analysts         0.006609
building         0.006575
quantitative     0.006403
generate         0.006400
interpret        0.006393
understanding    0.006339
datasets         0.005927
integrate        0.005793
environments     0.005661
learning         0.005584
sequencing       0.004943
